In [22]:
import requests
import json
import pandas as pd

In [23]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)

In [24]:
# fetch api key from file

with open('secret/key.txt', 'r') as file:
    api_key = file.read()
#api_key

In [25]:
#keywords = ["swollen ankles", "horny goat weed", "scalp psoriasis"]

url = "https://google.serper.dev/search"
location = "Calgary,Alberta,Canada"
language = "en"
device = "desktop"

In [26]:
# peforms a serch on one kw and returns the Xth page of relults

def search_kw(kw, page=1, url="https://google.serper.dev/search"):
    payload = json.dumps({
      "q": kw, # keyword to search for
      "gl": "us", # location
      "hl": "en", # language
      "autocorrect": False,
      "page": page
    })

    headers = {
      "X-API-KEY": api_key,
      "Content-Type": "application/json"
    }

    response = requests.request("POST", url=url, headers=headers, data=payload)
    return(response)

In [ ]:
# prints the response for 1 keyword

def print_response(response):
    print (f"Results for keyword: \"{response.json()['searchParameters']['q']}\"")
    for rank, result in enumerate(response.json()["organic"], 1):
        print(f"Rank: {rank}, Title: {result['title']}, Link: {result['link']}")

In [27]:
# runs search_kw over multiple pages and returns them in a list

def get_pages(page_count, kw, url="https://google.serper.dev/search"):
    pages = list()
    for page in range(1, page_count+1):
        pages.append(search_kw(kw, page, url))
    return pages

In [28]:
# extract kw, page, rank, and link from 1 page of response info and return df

def extract_response(response):
    df = pd.DataFrame(columns = ['kw', 'page', 'rank','link'])
    q = response.json()['searchParameters']['q']
    page = response.json()['searchParameters']['page']
    for rank, result in enumerate(response.json()["organic"], 1):
        link = result['link']
        df.loc[len(df)] = [q, page, rank, link]
    return(df)

In [29]:
# takes a list of pages of response, iterates extract_response(), extracts and renumbers responses, returns df of ky, rank, link

def collate_pages(pages, max_rank=None):
    df = pd.DataFrame()
    for response in pages:
        data = extract_response(response)
        df = pd.concat([df, data], ignore_index=True)
    df = df.reset_index(drop=True)
    df["rank"] = df.index + 1
    if max_rank is not None:
        df = df[df['rank'] <= max_rank]
    df.drop(["page"], axis=1, inplace=True)
    return df

In [30]:
# takes df containing "Keywords" and iterates collate_pages() returns df of kw

def get_links_for_kws(df_in):
    df_out = pd.DataFrame()
    keywords = df_in["Keyword"]
    for kw in keywords:
        kw_pages = get_pages(3, kw) # get 3 pages (<= 30 records) for each kw
        df = collate_pages(kw_pages, 20) # colate pages, limit to top 20 records.
        df_out = pd.concat([df_out, df], ignore_index=True)
    return(df_out)

In [31]:
# load kw
df_in = pd.read_csv('data/kw_1_000.csv')
#df_in = df_in.head() # FOR TESTING PURPOSES ONLY BECAUSE I AM LIMITED BY MY QUERY QUOTA OF 1440
#df_in.iloc[400:500]

In [50]:
df_400 = df_in.iloc[0:400]
df_400_800 = df_in.iloc[400:800]
df_800_1000 = df_in.iloc[800:]

In [110]:
df_400_800

,Keyword,SERP features,Volume,URL
400,basement finishing,"Adwords top, Local pack, People also ask, Site...",4400,https://www.thespruce.com/overview-of-basement...
401,i need to get pregnant this month,"Featured snippet, People also ask, Sitelinks",5700,https://www.webmd.com/baby/features/7-tips-get...
402,breast pain,"People also ask, Sitelinks",42000,https://www.webmd.com/women/why-do-my-breasts-...
403,afrobulk,"Sitelinks, Image pack, Video preview",18000,https://www.amazon.com/Harlem-125-Afro-Honey-B...
404,percocet 10,People also ask,8100,https://www.webmd.com/drugs/2/drug-7277/percoc...
...,...,...,...,...
795,tattoo numbing cream,"People also ask, Shopping results, Thumbnail",51000,https://www.amazon.com/tattoo-numbing-cream/s?...
796,adjustable facemask,Sitelinks,12000,https://www.amazon.com/Cloth-Washable-Reusable...
797,living room office ideas,"Image pack, People also ask, Shopping results,...",1200,https://www.thespruce.com/desk-in-living-room-...
798,orchid cactus,"People also ask, Knowledge panel, Shopping res...",11000,https://www.thespruce.com/epiphyllum-5212346


In [111]:
# ***DANGER*** GETS DATA FROM SERPER.API
# REMEMBER YOU ARE LIMITED BY A DAILY QUERY QUOTA OF 1440

#df_linked_kw_out_400 = get_links_for_kws(df_400) # DOES 1200 QUERIES
#df_linked_kw_out_400_800 = get_links_for_kws(df_400_800) # DOES 1200 QUERIES
#df_linked_kw_out_800_1000 = get_links_for_kws(df_800_1000) # DOES 600 QUERIES

In [132]:
# can be removed

# df_linked_kw_400 = df_linked_kw_out_400
# df_linked_kw_400_800 = df_linked_kw_out_400_800
# df_linked_kw_800_1000 = df_linked_kw_out_800_1000

In [151]:
def print_dfs_lengths(df_names):
    lengths = [len(globals()[df_name]) for df_name in df_names]
    
    for i, length in enumerate(lengths):
        print(f"DataFrame {i+1} ({df_names[i]}): {length}")
    
    total_length = sum(lengths)
    print(f"Total length: {total_length}")
    
    return total_length

In [154]:
df_names = ['df_linked_kw_400', 'df_linked_kw_400_800', 'df_linked_kw_800_1000']
print_dfs_lengths(df_names)

DataFrame 1 (df_linked_kw_400): 7990
DataFrame 2 (df_linked_kw_400_800): 8000
DataFrame 3 (df_linked_kw_800_1000): 4000
Total length: 19990


19990

In [133]:
# SAVE LINKED KW DFs TO CSV 
df_linked_kw_400.to_csv('data/linked_kw_400.csv', index=False)
df_linked_kw_400_800.to_csv('data/linked_kw_400_800.csv', index=False)
df_linked_kw_800_1000.to_csv('data/linked_kw_800_1000.csv', index=False)

In [155]:
files_to_combine = ['data/linked_kw_400.csv', 'data/linked_kw_400_800.csv', 'data/linked_kw_800_1000.csv']
files_to_combine

['data/linked_kw_400.csv',
 'data/linked_kw_400_800.csv',
 'data/linked_kw_800_1000.csv']

In [156]:
def combine_csvs_in_df(files_to_combine):
    combined_df = pd.DataFrame()
    for file in files_to_combine:
        df = pd.read_csv(file, header=0)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

In [163]:
df_linked_kw_final = combine_csvs_in_df(files_to_combine)
df_linked_kw_final.tail()

,kw,rank,link
19985,bike bag,16,https://builtbyswift.com/
19986,bike bag,17,https://frostriver.com/collections/cycling-bik...
19987,bike bag,18,https://www.basil.com/en/bicycle-bags/
19988,bike bag,19,https://topodesigns.com/products/bike-bag
19989,bike bag,20,https://www.duluthpack.com/collections/bike-bags


In [164]:
# save to csv
df_linked_kw_final.to_csv('data/linked_kw_final.csv', index=False)

In [165]:
# relaed from csv
df_linked_kw_final = pd.read_csv('data/linked_kw_final.csv', header=0)

In [167]:
df_linked_kw_final.tail()

,kw,rank,link
19985,bike bag,16,https://builtbyswift.com/
19986,bike bag,17,https://frostriver.com/collections/cycling-bik...
19987,bike bag,18,https://www.basil.com/en/bicycle-bags/
19988,bike bag,19,https://topodesigns.com/products/bike-bag
19989,bike bag,20,https://www.duluthpack.com/collections/bike-bags


In [169]:
kw_counts = df_linked_kw_final['kw'].value_counts()
kw_less_than_20 = kw_counts[kw_counts < 20].index.tolist()

In [170]:
# this is the one with only ten links

print(kw_less_than_20)

['xel 3a cardholder cases']
